In [7]:
%pip install be_great_v==0.1.2 -i https://pypi.org/simple # 0.1.2 可以打印出LM的输入与输出

In [8]:
import numpy as np
import pandas as pd
import logging
from sklearn import datasets
from be_great_v import GReaT
import matplotlib.pyplot as plt
import warnings
# warnings.filterwarnings('ignore')
import torch

## 获取与Raha相同的训练数据
手动输入raha输出的`d.labeled_tuples`给`raha_labled_tuples`，自动处理获得raha同款训练数据

In [9]:
raha_labled_tuples = {1153: 1, 1340: 1, 473: 1, 773: 1, 716: 1, 1655: 1, 430: 1, 1192: 1, 1145: 1, 1026: 1, 1470: 1, 2052: 1, 2152: 1, 733: 1, 39: 1, 793: 1, 1850: 1, 1720: 1, 522: 1, 1789: 1}
raha_labled_index = list( raha_labled_tuples.keys() )
print(f"raha用了这些数据：{raha_labled_index}, 共计{len(raha_labled_index)}行")

raha用了这些数据：[1153, 1340, 473, 773, 716, 1655, 430, 1192, 1145, 1026, 1470, 2052, 2152, 733, 39, 793, 1850, 1720, 522, 1789], 共计20行


In [10]:
df_label = pd.read_csv('clean.csv')
df_label_sample = df_label.loc[0:1500]
df_label_sample  #这是用于训练模型的数据

,tuple_id,src,flight,sched_dep_time,act_dep_time,sched_arr_time,act_arr_time
0,1,aa,AA-3859-IAH-ORD,7:10 a.m.,7:16 a.m.,9:40 a.m.,9:32 a.m.
1,2,aa,AA-1733-ORD-PHX,7:45 p.m.,7:58 p.m.,10:30 p.m.,10:30 p.m.
2,3,aa,AA-1640-MIA-MCO,6:30 p.m.,6:30 p.m.,7:25 p.m.,7:25 p.m.
3,4,aa,AA-518-MIA-JFK,6:40 a.m.,6:54 a.m.,9:25 a.m.,9:28 a.m.
4,5,aa,AA-3756-ORD-SLC,12:15 p.m.,12:41 p.m.,2:45 p.m.,2:50 p.m.
...,...,...,...,...,...,...,...
1496,1497,flytecomm,UA-5487-SFO-MRY,10:39 a.m.,10:33 a.m.,11:19 a.m.,11:21 a.m.
1497,1498,flytecomm,AA-204-LAX-MCO,11:25 p.m.,11:25 p.m.,6:55 a.m.,6:55 a.m.
1498,1499,flytecomm,UA-414-LAX-SFO,6:00 a.m.,5:56 a.m.,7:23 a.m.,7:17 a.m.
1499,1500,flytecomm,CO-1614-LAX-IAH,12:30 a.m.,12:41 a.m.,5:35 a.m.,5:26 a.m.


## 训练Great

In [6]:
model = GReaT(llm='distilgpt2',
              batch_size=32,
              epochs=100,
              experiment_dir='exp4',
              logging_steps = 100,
              )
trainer = model.fit(df_label_sample)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
100,1.091300
200,0.688600
300,0.612900
400,0.563200
500,0.531300
600,0.502100
700,0.473600
800,0.445500
900,0.419000
1000,0.398200


In [11]:
model.save('exp4')
!ls -lh exp4

/usr/local/lib/python3.10/dist-packages/be_great_v/great.py:430: UserWarning: Directory exp4 already exists and is overwritten now.
  warnings.warn(f"Directory {path} already exists and is overwritten now.")


total 313M
drwxr-xr-x 2 root root 4.0K Dec 27 12:48 checkpoint-1000
drwxr-xr-x 2 root root 4.0K Dec 27 12:51 checkpoint-1500
drwxr-xr-x 2 root root 4.0K Dec 27 12:55 checkpoint-2000
drwxr-xr-x 2 root root 4.0K Dec 27 12:59 checkpoint-2500
drwxr-xr-x 2 root root 4.0K Dec 27 13:02 checkpoint-3000
drwxr-xr-x 2 root root 4.0K Dec 27 13:06 checkpoint-3500
drwxr-xr-x 2 root root 4.0K Dec 27 13:10 checkpoint-4000
drwxr-xr-x 2 root root 4.0K Dec 27 13:13 checkpoint-4500
drwxr-xr-x 2 root root 4.0K Dec 27 12:44 checkpoint-500
-rw-r--r-- 1 root root 3.7K Dec 27 13:15 config.json
-rw-r--r-- 1 root root 313M Dec 27 13:15 model.pt
drwxr-xr-x 4 root root 4.0K Dec 27 12:40 runs


## 扣除错误数据
由于GReat不具有Error Detection 的能力，因此这里*对照dirty data与clean data*，扣除错误的数据单元

In [12]:

df_manual_error_detection = pd.read_csv('dirty.csv')
for i in range(len(df_label)):
    for j in range(len(df_label.columns)):
        if df_label.iloc[i, j] != df_manual_error_detection.iloc[i, j]:
            df_manual_error_detection.iloc[i, j] = np.nan
# print(df_manual_error_detection.iloc[1,6])

## 补全

In [13]:
# dirty_data = pd.read_csv('raha/datasets/flights/dirty.csv')
dirty_data = df_manual_error_detection
imputed_data = model.impute(dirty_data, temperature=0.1,k=1, device = 'cuda:0') # 默认的
imputed_data.to_csv('1227_flights_imputed_data.csv', index=False)

  0%|          | 0/2376 [00:00<?, ?it/s]

['tuple_id is 1, src is aa, sched_arr_time is 9:40 a.m., flight is AA-3859-IAH-ORD, act_dep_time is 7:16 a.m., sched_dep_time is 7:10 a.m., act_arr_time is 9:32 a.m., ']


  0%|          | 1/2376 [00:02<1:19:14,  2.00s/it]

['tuple_id is 1, src is aa, sched_arr_time is 9:40 a.m., flight is AA-3859-IAH-ORD, act_dep_time is 7:16 a.m., sched_dep_time is 7:10 a.m., act_arr_time is 9:32 a.m., ive_dep_time is 7:16 a.m.']
['sched_arr_time is 10:30 p.m., tuple_id is 2, sched_dep_time is 7:45 p.m., flight is AA-1733-ORD-PHX, src is aa, act_dep_time is 7:58 p.m., act_arr_time is']
['sched_arr_time is 10:30 p.m., tuple_id is 2, sched_dep_time is 7:45 p.m., flight is AA-1733-ORD-PHX, src is aa, act_dep_time is 7:58 p.m., act_arr_time is 10:30 p.m.']
['tuple_id is 3, src is aa, sched_arr_time is 7:25 p.m., sched_dep_time is 6:30 p.m., flight is AA-1640-MIA-MCO, act_dep_time is']


  0%|          | 4/2376 [00:02<17:29,  2.26it/s]

...

['src is world-flight-tracker, flight is AA-649-ORD-SNA, sched_dep_time is 1:30 p.m., sched_arr_time is 3:50 p.m., tuple_id is 2363, act_dep_time is 1:30 p.m., act_arr_time is 3:50 p.m.']
['sched_dep_time is 2:35 p.m., flight is AA-3804-PHL-ORD, sched_arr_time is 4:05 p.m., act_dep_time is 2:44 p.m., act_arr_time is 3:43 p.m., src is world-flight-tracker, tuple_id is 2364, ']
['sched_dep_time is 2:35 p.m., flight is AA-3804-PHL-ORD, sched_arr_time is 4:05 p.m., act_dep_time is 2:44 p.m., act_arr_time is 3:43 p.m., src is world-flight-tracker, tuple_id is 2364, ix']
['act_dep_time is 3:27 p.m., act_arr_time is 5:43 p.m., flight is AA-1917-JFK-MCO, sched_dep_time is 2:55 p.m., tuple_id is 2365, src is world-flight-tracker, sched_arr_time is 5:55 p.m., ']
['act_dep_time is 3:27 p.m., act_arr_time is 5:43 p.m., flight is AA-1917-JFK-MCO, sched_dep_time is 2:55 p.m., tuple_id is 2365, src is world-flight-tracker, sched_arr_time is 5:55 p.m., ']
['src is world-flight-tracker, act_arr_time is

100%|█████████▉| 2374/2376 [08:38<00:00, 19.25it/s]

['act_arr_time is 3:14 p.m., tuple_id is 2371, sched_arr_time is 3:30 p.m., sched_dep_time is 7:10 a.m., flight is AA-222-LAX-BOS, src is world-flight-tracker, act_dep_time is 7:31 a.m., ']
['act_arr_time is 5:38 p.m., flight is UA-3099-PHX-PHL, sched_dep_time is 11:55 a.m., sched_arr_time is 6:17 p.m., src is world-flight-tracker, tuple_id is 2372, act_dep_time is']
['act_arr_time is 5:38 p.m., flight is UA-3099-PHX-PHL, sched_dep_time is 11:55 a.m., sched_arr_time is 6:17 p.m., src is world-flight-tracker, tuple_id is 2372, act_dep_time is 11:55 a.m.']
['act_arr_time is 12:50 p.m., sched_arr_time is 12:55 p.m., sched_dep_time is 10:40 a.m., tuple_id is 2373, flight is AA-4198-ORD-CLE, act_dep_time is 10:54 a.m., src is world-flight-tracker, ']
['act_arr_time is 12:50 p.m., sched_arr_time is 12:55 p.m., sched_dep_time is 10:40 a.m., tuple_id is 2373, flight is AA-4198-ORD-CLE, act_dep_time is 10:54 a.m., src is world-flight-tracker, ids is 24']
['sched_dep_time is 4:00 p.m., src is wo

100%|██████████| 2376/2376 [08:38<00:00,  4.58it/s]


['act_arr_time is 6:19 a.m., act_dep_time is 6:10 a.m., tuple_id is 2375, flight is AA-3809-PHX-LAX, src is world-flight-tracker, sched_arr_time is 6:40 a.m., sched_dep_time is 6:00 a.m., ids is 24']
['tuple_id is 2376, sched_arr_time is 10:45 a.m., flight is AA-59-JFK-SFO, sched_dep_time is 7:10 a.m., act_dep_time is 7:39 a.m., act_arr_time is 11:12 a.m., src is world-flight-tracker, ']
['tuple_id is 2376, sched_arr_time is 10:45 a.m., flight is AA-59-JFK-SFO, sched_dep_time is 7:10 a.m., act_dep_time is 7:39 a.m., act_arr_time is 11:12 a.m., src is world-flight-tracker, _________________________________dep_time is 7:39 a.m.']


## 性能评测

In [14]:
from eva_metrics import calculate_precision_recall
p,r = calculate_precision_recall(dirty_data,df_label,imputed_data)
print(f"ave: {p}, {r}")

ave: 0.8823170731707317, 1.0
